## Cifar-10 CNN
### Import, init

In [ ]:
import keras
import numpy as np
import pickle

keras.backend.set_image_data_format('channels_first')
print('Using data format:', keras.backend.image_data_format())

### Read data

In [ ]:
img_row, img_col, img_cnl = 32, 32, 3
label_names = { }
def read_imgs(filename):
    f = open(filename, 'rb')
    data = pickle.load(f, encoding='bytes')
    f.close()
    imgs = np.array(data[b'data'])
    imgs = imgs.reshape((imgs.shape[0], img_cnl, img_row, img_col))
    labels = np.array(data[b'labels'])
    return imgs, labels
def load_labels(filename):
    f = open(filename, 'rb')
    data = pickle.load(f, encoding='bytes')
    f.close()
    return data[b'label_names']
print('# test: ')
%time imgs, labels = read_imgs('datas/data_batch_1')
print('# data shape:' ,imgs.shape)
print('# label names')
print(load_labels('datas/batches.meta'))

read all data from file

In [ ]:
# Load datas
train_x, train_y = read_imgs('datas/data_batch_1')
for x in (2, 3, 4, 5):
    fn = 'datas/data_batch_%d' % x
    imgs, labels = read_imgs(fn)
    train_x = np.concatenate((train_x, imgs))
    train_y = np.concatenate((train_y, labels))
test_x, test_y = read_imgs('datas/test_batch')

# Load label names
label_names = load_labels('datas/batches.meta')
label_names = [x.decode('ascii') for x in label_names]

# Categorical
class_cnt = len(label_names)
train_y = keras.utils.to_categorical(train_y, class_cnt)
test_y = keras.utils.to_categorical(test_y, class_cnt)

# Standard
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
train_x /= 255
test_x /= 255

print('# Train data loaded:', train_x.shape[0])
print('# Test data loaded:', test_x.shape[0])
print('# Label names:')
print(label_names)

## Define network structure
Conv -> Conv -> Pooling -> Conv -> Conv -> Pooling -> Dense

In [ ]:
batch_size = 128
epochs = 300

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=train_x.shape[1:]))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(class_cnt, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train and evaluate

In [ ]:
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_x, test_y))
model.evaluate(test_x, test_y, verbose=0)

## Save the model

In [ ]:
model.save('trained_model2.h5')